In [1]:
import numpy as np
from my_stuff import MufexKeys
from quantfreedom.enums import CandleProcessingType
from quantfreedom.exchanges.mufex_exchange.mufex import Mufex
from quantfreedom.base import backtest_df_only
from quantfreedom.enums import *
from quantfreedom.helper_funcs import create_os_cart_product_nb
from quantfreedom.strategies.strategy_example import StrategyExample

mufex = Mufex(
    api_key=MufexKeys.api_key,
    secret_key=MufexKeys.secret_key,
    use_test_net=False
)
symbol = "BTCUSDT"
%load_ext autoreload
%autoreload 2

In [11]:
candles = mufex.get_candles_df(symbol=symbol, timeframe='5m', candles_to_dl=2000)

It took 00 mins and 03 seconds to download 2000 candles


In [12]:
mufex.set_exchange_settings(
    symbol=symbol,
    position_mode=PositionModeType.HedgeMode,
    leverage_mode=LeverageModeType.Isolated,
)
backtest_settings = BacktestSettings()
order_settings_arrays = OrderSettingsArrays(
    long_or_short=np.array([LongOrShortType.Long]),
    increase_position_type=np.array([IncreasePositionType.RiskPctAccountEntrySize]),
    risk_account_pct_size=np.array([1]),
    max_equity_risk_pct=np.array([3,10]),
    stop_loss_type=np.array([StopLossStrategyType.SLBasedOnCandleBody]),
    sl_candle_body_type=np.array([CandleBodyType.Low]),
    sl_based_on_add_pct=np.array([0.01]),
    sl_based_on_lookback=np.array([20]),
    sl_to_be_based_on_candle_body_type=np.array([CandleBodyType.Nothing]),
    sl_to_be_when_pct_from_candle_body=np.array([0.0]),
    sl_to_be_zero_or_entry_type=np.array([SLToBeZeroOrEntryType.Nothing]),
    trail_sl_based_on_candle_body_type=np.array([CandleBodyType.High]),
    trail_sl_when_pct_from_candle_body=np.array([3.0]),
    trail_sl_by_pct=np.array([1.0]),
    take_profit_type=np.array([TakeProfitStrategyType.RiskReward]),
    risk_reward=np.array([1,5]),
    tp_fee_type=np.array([TakeProfitFeeType.Limit]),
    leverage_type=np.array([LeverageStrategyType.Dynamic]),
    static_leverage=np.array([0.0]),
    num_candles=np.array([0]),
    entry_size_asset=np.array([0.0]),
    max_trades=np.array([3]),
    
)
os_cart_arrays = create_os_cart_product_nb(
    order_settings_arrays=order_settings_arrays,
)

In [14]:
strategy = StrategyExample(
    candle_processing_mode=CandleProcessingType.Backtest,
    candles=candles,
    rsi_length=[14, 30],
    rsi_is_below=[50,80],
)
candles_reg = backtest_df_only(
    starting_equity=1000.0,
    os_cart_arrays=os_cart_arrays,
    backtest_settings=backtest_settings,
    exchange_settings=mufex.exchange_settings,
    strategy=strategy,
)

Starting the backtest now ... and also here are some stats for your backtest.

Total indicator settings to test: 4
Total order settings to test: 4
Total combinations of settings to test: 16

Total candles: 2,000
Total candles to test: 32,000


In [9]:
candles_reg.sort_values(['ind_set_idx','or_set_idx']).iloc[:]

,ind_set_idx,or_set_idx,total_trades,gains_pct,win_rate,to_the_upside,total_pnl,ending_eq
14,0,0,38.0,-45.44,15.79,-9.849164e-01,-454.3828,545.6172
4,0,1,33.0,-43.17,9.09,-5.881924e-27,-431.7232,568.2768
9,0,2,36.0,-70.51,11.11,-9.376991e-01,-705.1268,294.8732
12,0,3,32.0,-71.05,9.38,-9.680710e-01,-710.5358,289.4642
13,1,0,38.0,-46.75,13.16,-9.807720e-01,-467.4955,532.5045
11,1,1,34.0,-43.41,8.82,-9.577708e-01,-434.0783,565.9217
6,1,2,37.0,-75.96,10.81,-1.489657e-26,-759.5759,240.4241
2,1,3,33.0,-75.93,9.09,-2.389999e-27,-759.2711,240.7289
5,2,0,35.0,-38.00,17.14,-1.040042e-26,-379.9678,620.0322
7,2,1,30.0,-33.57,10.00,-7.970596e-01,-335.7091,664.2909


In [10]:
strategy.indicator_cart_product

IndicatorSettingsArrays(rsi_length=array([14, 14, 30, 30]), rsi_is_below=array([50, 80, 50, 80]))